In [1]:
# =========================
# IMPORTS
# =========================

from X_functions import *
from X_config import *
from X_models import *
from X_interpolation import *
from X_products import *

from GGS_main import *

# =========================

c:\Users\sal_f\anaconda3\envs\env_ggs0\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---

### GGS Executable

In [2]:
power=1
path="local"
config_name="test"

In [3]:
if config_name is None:
    print("No config file specified. Exiting.")
    exit(1)

config = GGS_config_import(config_name)

target_datetime = config['MISSION'].get('target_date')
if not target_datetime:
    print("Issue with target datetime. Using current datetime.")
    target_datetime = dt.datetime.now(dt.timezone.utc)

if path == "local":
    root_directory = GGS_config_process(config, path="default")
elif path == "rucool":
    root_directory = GGS_config_process(config, path="/www/web/rucool/hurricane/model_comparisons/maps/yucatan")
else:
    raise ValueError("Invalid root directory.")

datetime_list = [target_datetime.replace(hour=0, minute=0, second=0, microsecond=0).strftime('%Y-%m-%dT%H:%M:%SZ')]
datetime_index = datetime_list[0]

glider_dataframes = None
if config['PRODUCT'].get('show_gliders'):
    min_lat, min_lon = config['MISSION']['extent'][0]
    max_lat, max_lon = config['MISSION']['extent'][1]
    search_extent = [min_lon, max_lon, min_lat, max_lat]
    glider_dataframes = acquire_gliders(
        extent=search_extent,
        target_date=target_datetime,
        date_delta=dt.timedelta(days=1),
        requested_variables=["time", "longitude", "latitude", "profile_id", "depth"],
        print_vars=False,
        target="all",
        request_timeout=5,
        enable_parallel=False
    )


### IMPORTING GGS CONFIGURATION: test ###

Configuration import success!

### PROCESSING GGS CONFIGURATION ###

Configuration:

--> MISSION
mission_name: Sentinel Leg 1
target_date: 2024-04-11T20:17:59+0000
max_depth: 1000
extent: ((10, -78), (44, -10))
GPS_coords: [[41.675, -70.522], [40.04, -70.67], [37.875, -70.742], [37.738, -65.998], [39.12, -61.486], [36.026, -60.242], [31.012, -54.977], [31.79, -53.372], [26.176, -45.272], [20.712, -33.73], [15.067, -23.65]]
glider_id: None
glider_buffer: None

--> MODEL
single_datetime: True
enable_rtofs: False
enable_cmems: True
enable_gofs: True
chunk: True
save_model_data: False
save_depth_average: True
save_bin_average: False

--> PRODUCT
create_magnitude_plot: False
create_threshold_plot: True
create_advantage_plot: False
create_profile_plot: False
create_gpkg_file: False
latitude_qc: 39
longitude_qc: -69
density: 3
mag1: 0.0
mag2: 0.2
mag3: 0.3
mag4: 0.4
mag5: 0.5
tolerance: 15
show_gliders: False
show_waypoints: True
show_eez: True
show

In [4]:
datetime_index = datetime_index
config_flag = config
root_directory_flag = root_directory
glider_data_flag = glider_dataframes

enable_rtofs_flag = config_flag['MODEL']['enable_rtofs']
enable_cmems_flag = config_flag['MODEL']['enable_cmems']
enable_gofs_flag = config_flag['MODEL']['enable_gofs']
save_model_data_flag = config_flag['MODEL']['save_model_data']
save_depth_average_flag = config_flag['MODEL']['save_depth_average']
save_bin_average_flag = config_flag['MODEL']['save_bin_average']
chunk_flag = config_flag['MODEL']['chunk']

create_magnitude_plot_flag = config_flag['PRODUCT']['create_magnitude_plot']
create_threshold_plot_flag = config_flag['PRODUCT']['create_threshold_plot']
create_advantage_plot_flag = config_flag['PRODUCT']['create_advantage_plot']
create_profiles_plot_flag = config_flag['PRODUCT']['create_profile_plot']
create_gpkg_file_flag = config_flag['PRODUCT']['create_gpkg_file']
latitude_qc_flag = config_flag['PRODUCT']['latitude_qc']
longitude_qc_flag = config_flag['PRODUCT']['longitude_qc']
density_flag = config_flag['PRODUCT']['density']
mag1_flag = config_flag['PRODUCT']['mag1']
mag2_flag = config_flag['PRODUCT']['mag2']
mag3_flag = config_flag['PRODUCT']['mag3']
mag4_flag = config_flag['PRODUCT']['mag4']
mag5_flag = config_flag['PRODUCT']['mag5']
tolerance_flag = config_flag['PRODUCT']['tolerance']
show_waypoints_flag = config_flag['PRODUCT']['show_waypoints']
show_eez_flag = config_flag['PRODUCT']['show_eez']
show_qc_flag = config_flag['PRODUCT']['show_qc']
manual_extent_flag = config_flag['PRODUCT']['manual_extent']
compute_optimal_path_flag = config_flag['PRODUCT']['compute_optimal_path']

sub_directory_plots = os.path.join(root_directory_flag, "plots", ''.join(datetime_index[:10].split('-')))
os.makedirs(sub_directory_plots, exist_ok=True)
sub_directory_data = os.path.join(root_directory_flag, "data", ''.join(datetime_index[:10].split('-')))
os.makedirs(sub_directory_data, exist_ok=True)

check_datetime = pd.to_datetime(datetime_index).strftime('%Y%m%dT%HZ')
check_pattern = os.path.join(sub_directory_data, f"*_DepthAverageData_{check_datetime}.nc")
check_files = glob.glob(check_pattern)
if check_files:
    print(f"Datetime {datetime_index} already processed: {check_files[0]}, skipping task.")
else:
    print(f"Datetime {datetime_index} unprocessed, proceeding with task.")

Datetime 2024-04-11T00:00:00Z unprocessed, proceeding with task.


In [5]:
model_datasets = []

if config_flag['ADVANCED']['reprocess']:
    current_directory = os.getcwd()
    reprocess_path = os.path.join(current_directory, "data/reprocess")

    model_datasets = []
    model_files = glob.glob(os.path.join(reprocess_path, '*.nc'))
    for model_file in model_files:
        depth_average_dataset = xr.open_dataset(model_file)
        model_name = depth_average_dataset.attrs['model_name']
        if model_name == 'RTOFS':
            rtofs_datasets = (None, depth_average_dataset, None)
            model_datasets.append(rtofs_datasets)
        elif model_name == 'CMEMS':
            cmems_datasets = (None, depth_average_dataset, None)
            model_datasets.append(cmems_datasets)
        elif model_name == 'GOFS':
            gofs_datasets = (None, depth_average_dataset, None)
            model_datasets.append(gofs_datasets)
else:
    if enable_rtofs_flag:
        try:
            rtofs = RTOFS()
            rtofs.rtofs_load(config_flag, datetime_index)
            rtofs.rtofs_save(config_flag, sub_directory_data, save_data=save_model_data_flag)
            rtofs_model_data = rtofs.data
            rtofs_depth_average, rtofs_bin_average = interpolate_rtofs(config_flag, sub_directory_data, rtofs_model_data, chunk=chunk_flag, save_depth_average=save_depth_average_flag, save_bin_average=save_bin_average_flag)
            rtofs_datasets = (rtofs_model_data, rtofs_depth_average, rtofs_bin_average)
            model_datasets.append(rtofs_datasets)
        except Exception as e:
            print(f"Error during RTOFS processing: {e}")
    if enable_cmems_flag:
        try:
            cmems = CMEMS(username='sfricano1', password='GlobalGliders1')
            cmems.cmems_load(config_flag, datetime_index)
            cmems.cmems_save(config_flag, sub_directory_data, save_data=save_model_data_flag)
            cmems_model_data = cmems.data
            cmems_depth_average, cmems_bin_average = interpolate_cmems(config_flag, sub_directory_data, cmems_model_data, chunk=chunk_flag, save_depth_average=save_depth_average_flag, save_bin_average=save_bin_average_flag)
            cmems_datasets = (cmems_model_data, cmems_depth_average, cmems_bin_average)
            model_datasets.append(cmems_datasets)
        except Exception as e:
            print(f"Error during CMEMS processing: {e}")
    if enable_gofs_flag:
        try:
            gofs = GOFS()
            gofs.gofs_load(config_flag, datetime_index)
            gofs.gofs_save(config_flag, sub_directory_data, save_data=save_model_data_flag)
            gofs_model_data = gofs.data
            gofs_depth_average, gofs_bin_average = interpolate_gofs(config_flag, sub_directory_data, gofs_model_data, chunk=chunk_flag, save_depth_average=save_depth_average_flag, save_bin_average=save_bin_average_flag)
            gofs_datasets = (gofs_model_data, gofs_depth_average, gofs_bin_average)
            model_datasets.append(gofs_datasets)
        except Exception as e:
            print(f"Error during GOFS processing: {e}")

In [6]:
if create_magnitude_plot_flag:
    GGS_plot_magnitude(
        config_flag,
        sub_directory_plots,
        datetime_index,
        model_datasets,
        latitude_qc=latitude_qc_flag, longitude_qc=longitude_qc_flag,
        density=density_flag,
        gliders=glider_data_flag,
        show_waypoints=show_waypoints_flag, show_eez=show_eez_flag, show_qc=show_qc_flag,
        manual_extent=manual_extent_flag,
        compute_optimal_path=compute_optimal_path_flag
    )
if create_threshold_plot_flag:
    GGS_plot_threshold(
        config_flag,
        sub_directory_plots,
        datetime_index,
        model_datasets,
        latitude_qc=latitude_qc_flag, longitude_qc=longitude_qc_flag,
        density=density_flag,
        mag1=mag1_flag, mag2=mag2_flag, mag3=mag3_flag, mag4=mag4_flag, mag5=mag5_flag,
        gliders=glider_data_flag,
        show_waypoints=show_waypoints_flag, show_eez=show_eez_flag, show_qc=show_qc_flag,
        manual_extent=manual_extent_flag,
        compute_optimal_path=compute_optimal_path_flag
    )
if create_advantage_plot_flag:
    GGS_plot_advantage(
        config_flag,
        sub_directory_plots,
        datetime_index,
        model_datasets,
        latitude_qc=latitude_qc_flag, longitude_qc=longitude_qc_flag,
        density=density_flag,
        tolerance=tolerance_flag,
        mag1=mag1_flag, mag2=mag2_flag, mag3=mag3_flag, mag4=mag4_flag, mag5=mag5_flag,
        gliders=glider_data_flag,
        show_waypoints=show_waypoints_flag, show_eez=show_eez_flag, show_qc=show_qc_flag,
        manual_extent=manual_extent_flag,
        compute_optimal_path=compute_optimal_path_flag
    )
if create_profiles_plot_flag:
    GGS_plot_profiles(
        config_flag,
        sub_directory_plots,
        datetime_index,
        model_datasets,
        latitude_qc=latitude_qc_flag, longitude_qc=longitude_qc_flag,
        threshold=0.5
    )
if create_gpkg_file_flag:
    GGS_export_gpkg(
        sub_directory_data,
        datetime_index,
        model_datasets
    )


### CREATING THRESHOLD PLOT ###

Start time (UTC): 2024-04-11 20:17:59.561396
Computing optimal path for the mission.
!!!WARNING!!!: This feature is experimental and may not work as expected. Use at your own risk.
Direct path used from (41.66667, -70.5) to (40.0, -70.666664).
Total mission time (adjusted): [17688631.84835901] seconds
Total mission distance: 6098654.257977845 meters
End time (UTC): 2024-04-11 20:25:20.823584
Run time: 0:07:21.262188


---

### GGS Testing

In [10]:
def compute_optimal_path(config, model_dataset, glider_raw_speed=0.5):
    
    '''
    Calculates the optimal path between waypoints for a mission, considering the impact of ocean currents and distance.
    
    This function uses the "A*" algorithm to determine the most efficient path between waypoints specified in the config,
    taking into account the ocean's depth-averaged current data provided by the "model_dataset".

    Args:
    - config (dict): A dictionary containing mission config details including waypoints.
    - model_dataset (xarray.Dataset): An xarray dataset containing depth-averaged ocean current data.
    - glider_raw_speed (float, optional): The glider's base speed in meters per second
        - default: 0.5

    Returns:
    - optimal_mission_path (list of tuples): A list of latitude and longitude tuples representing the optimal route.
    '''
    
    print("Computing optimal path for the mission.")
    print("!!!WARNING!!!: This feature is experimental and may not work as expected. Use at your own risk.")

    def calculate_haversine_distance(longitude1, latitude1, longitude2, latitude2):
        '''Calculates the great circle distance between two points on the earth using the Haversine formula.'''
        longitude1, latitude1, longitude2, latitude2 = map(radians, [longitude1, latitude1, longitude2, latitude2])
        delta_longitude = longitude2 - longitude1
        delta_latitude = latitude2 - latitude1
        a = sin(delta_latitude / 2)**2 + cos(latitude1) * cos(latitude2) * sin(delta_longitude / 2)**2
        distance = 2 * asin(sqrt(a)) * 6371000
        return distance
    
    def calculate_route_analytics(model_dataset, start_index, end_index, glider_raw_speed):
        '''Calculates time, distance, and adjusted time between two points considering ocean currents.'''
        start_lat, start_lon = convert_grid2coord(*start_index)
        end_lat, end_lon = convert_grid2coord(*end_index)
        direction = np.arctan2(end_lon - start_lon, end_lat - start_lat)
        u_current = model_dataset['u_depth_avg'].isel(lat=start_index[0], lon=start_index[1]).values
        v_current = model_dataset['v_depth_avg'].isel(lat=start_index[0], lon=start_index[1]).values
        current_speed = u_current * np.cos(direction) + v_current * np.sin(direction)
        net_speed = max(glider_raw_speed + current_speed, 0.1)
        distance = calculate_haversine_distance(start_lon, start_lat, end_lon, end_lat)
        time = distance / net_speed
        return time, distance

    def calculate_direct_path(start_index, end_index, glider_raw_speed):
        '''Fallback to the direct great circle path if no optimal path is found.'''
        start_lat, start_lon = convert_grid2coord(*start_index)
        end_lat, end_lon = convert_grid2coord(*end_index)
        distance = calculate_haversine_distance(start_lon, start_lat, end_lon, end_lat)
        time = distance / glider_raw_speed
        return [(start_lat, start_lon), (end_lat, end_lon)], time, distance

    def convert_coord2grid(latitude, longitude):
        '''Converts geographical latitude and longitude to the nearest index on the dataset grid.'''
        latitude_index = np.argmin(np.abs(latitude_array - latitude))
        longitude_index = np.argmin(np.abs(longitude_array - longitude))
        return latitude_index, longitude_index
    
    def convert_grid2coord(latitude_index, longitude_index):
        '''Converts dataset grid indices back to geographical latitude and longitude coordinates.'''
        latitude = latitude_array[latitude_index]
        longitude = longitude_array[longitude_index]
        return latitude, longitude
    
    def calculate_heuristic_cost(current_index, goal_index):
        '''Estimates the cost from the current index to the goal using the Haversine formula as a heuristic.'''
        current_latitude, current_longitude = convert_grid2coord(*current_index)
        goal_latitude, goal_longitude = convert_grid2coord(*goal_index)
        heuristic_cost = calculate_haversine_distance(current_longitude, current_latitude, goal_longitude, goal_latitude)
        return heuristic_cost
    
    def calculate_movement_cost(model_dataset, current_index, next_index, speed):
        '''Calculates the cost of moving from the current index to the next, taking into account the effect of ocean currents.'''
        current_latitude, current_longitude = convert_grid2coord(*current_index)
        next_latitude, next_longitude = convert_grid2coord(*next_index)
        direction_to_next_index = np.arctan2(next_longitude - current_longitude, next_latitude - current_latitude)
        u_current_component = model_dataset['u_depth_avg'].isel(lat=current_index[0], lon=current_index[1]).values
        v_current_component = model_dataset['v_depth_avg'].isel(lat=current_index[0], lon=current_index[1]).values
        glider_velocity_component_u = speed * np.cos(direction_to_next_index)
        glider_velocity_component_v = speed * np.sin(direction_to_next_index)
        net_velocity_component_u = glider_velocity_component_u + u_current_component
        net_velocity_component_v = glider_velocity_component_v + v_current_component
        net_speed = np.sqrt(net_velocity_component_u**2 + net_velocity_component_v**2)
        net_speed = max(net_speed, 0.1)
        distance_to_next_index = calculate_haversine_distance(current_longitude, current_latitude, next_longitude, next_latitude)
        movement_cost = distance_to_next_index / net_speed
        return movement_cost
    
    def generate_neighbor_nodes(index):
        '''Generates neighboring index nodes for exploration based on the current index's position.'''
        latitude_index, longitude_index = index
        for delta_latitude in [-1, 0, 1]:
            for delta_longitude in [-1, 0, 1]:
                if delta_latitude == 0 and delta_longitude == 0:
                    continue
                new_latitude_index, new_longitude_index = latitude_index + delta_latitude, longitude_index + delta_longitude
                if 0 <= new_latitude_index < len(latitude_array) and 0 <= new_longitude_index < len(longitude_array):
                    yield (new_latitude_index, new_longitude_index)
    
    def reconstruct_path(came_from_dictionary, start_index, goal_index):
        '''Reconstructs the path from the start index to the goal index using the came_from dictionary populated by the A* algorithm.'''
        current_index = goal_index
        optimal_path = [current_index]
        while current_index != start_index:
            current_index = came_from_dictionary[current_index]
            optimal_path.append(current_index)
        optimal_path.reverse()
        optimal_path_coords = [convert_grid2coord(*index) for index in optimal_path]
        return optimal_path_coords
    
    def algorithm_a_star(model_dataset, start_index, end_index, glider_raw_speed):
        '''Executes the A* search algorithm to find the most efficient path from the start index to the goal index.'''
        open_set = [(calculate_heuristic_cost(start_index, end_index), start_index)]
        came_from = {start_index: None}
        g_score = {start_index: 0}
        f_score = {start_index: calculate_heuristic_cost(start_index, end_index)}
        path_found = False
        while open_set:
            _, current = heapq.heappop(open_set)
            if current == end_index:
                path_found = True
                break
            for neighbor in generate_neighbor_nodes(current):
                tentative_g_score = g_score[current] + calculate_movement_cost(model_dataset, current, neighbor, glider_raw_speed)
                if tentative_g_score < g_score.get(neighbor, float('inf')):
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = tentative_g_score + calculate_heuristic_cost(neighbor, end_index)
                    if neighbor not in [n for _, n in open_set]:
                        heapq.heappush(open_set, (f_score[neighbor], neighbor))
        if path_found:
            path = reconstruct_path(came_from, start_index, end_index)
            time, distance = calculate_route_analytics(model_dataset, start_index, end_index, glider_raw_speed)
        else:
            print(f"Direct path used from {convert_grid2coord(*start_index)} to {convert_grid2coord(*end_index)}.")
            path, time, distance = calculate_direct_path(start_index, end_index, glider_raw_speed)
        return path, time, distance
    
    print("New Version")

    mission_waypoints = config['MISSION']['GPS_coords']
    latitude_array = model_dataset['lat'].values
    longitude_array = model_dataset['lon'].values
    
    optimal_mission_path = []
    total_time = 0
    total_distance = 0
    
    for i in range(len(mission_waypoints) - 1):
        start_index = convert_coord2grid(*mission_waypoints[i])
        end_index = convert_coord2grid(*mission_waypoints[i + 1])
        segment_path, segment_time, segment_distance = algorithm_a_star(model_dataset, start_index, end_index, glider_raw_speed)
        optimal_mission_path.extend(segment_path[:-1])
        total_time += segment_time
        total_distance += segment_distance
    optimal_mission_path.append(mission_waypoints[-1])
    
    print(f"Total mission time (adjusted): {total_time} seconds")
    print(f"Total mission distance: [{total_distance}] meters")

    return optimal_mission_path

In [11]:
def plot_optimal_path(ax, config, model_depth_average):

    '''
    Plots the optimal path for the mission between all successive waypoints given in the GPS_coords list.

    Args:
    - ax (matplotlib.axes.Axes): Matplotlib axes object.
    - config (dict): Configuration dictionary.
    - model_depth_average (xarray.Dataset): Depth-averaged ocean current dataset.

    Returns:
    - None
    '''

    optimal_mission_path = compute_optimal_path(config, model_depth_average, glider_raw_speed=0.4)
    route_lats, route_lons = zip(*optimal_mission_path)
    
    ax.plot(route_lons, route_lats, 'o-', transform=ccrs.PlateCarree(), markersize=5, linewidth=3, color='black', zorder=94)

def GGS_plot_threshold(config, directory, datetime_index, model_datasets, latitude_qc=None, longitude_qc=None, density=2, mag1=0.0, mag2=0.2, mag3=0.3, mag4=0.4, mag5=0.5, gliders=None, show_waypoints=False, show_eez=False, show_qc=False, manual_extent=None, compute_optimal_path=False):
    
    '''
    Plot the depth-averaged current fields from three datasets side by side.

    Args:
    - config (dict): Glider Guidance System mission configuration.
    - directory (str): Directory to save the plot.
    - datetime_index (int): Index of the datetime for the plot title.
    - model_datasets (tuple): Tuple containing the model datasets.
    - latitude_qc (float): Latitude for QC plotting.
    - longitude_qc (float): Longitude for QC plotting.
    - density (int): Density of the streamplot.
    - mag1 (float): Threshold for the first magnitude level.
    - mag2 (float): Threshold for the second magnitude level.
    - mag3 (float): Threshold for the third magnitude level.
    - mag4 (float): Threshold for the fourth magnitude level.
    - mag5 (float): Threshold for the fifth magnitude level.
    - gliders (optional): DataFrame containing glider data for plotting.
    - show_waypoints (bool): Flag to show the glider waypoints.
    - show_qc (bool): Flag to show the QC sample point.
    - show_eez (bool): Flag to show the Exclusive Economic Zone (EEZ).
    - manual_extent (list or None): Manual specification of plot extent.
    - compute_optimal_path (bool): Flag to compute and plot the optimal path.

    Returns:
    - None
    '''

    print(f"\n### CREATING THRESHOLD PLOT ###\n")
    start_time = print_starttime()

    valid_datasets = [datasets for datasets in model_datasets if datasets is not None]
    num_datasets = len(valid_datasets)
    if num_datasets == 0:
        print("No datasets provided for plotting.")
        end_time = print_endtime()
        print_runtime(start_time, end_time)
        return

    def plot_threshold(ax, config, model_depth_average, latitude_qc, longitude_qc, density, mag1, mag2, mag3, mag4, mag5, gliders, show_waypoints, show_qc, show_eez, manual_extent, compute_optimal_path):
        
        longitude = model_depth_average.lon.values.squeeze()
        latitude = model_depth_average.lat.values.squeeze()
        u_depth_avg = model_depth_average['u_depth_avg'].values.squeeze()
        v_depth_avg = model_depth_average['v_depth_avg'].values.squeeze()
        mag_depth_avg = model_depth_average['mag_depth_avg'].values.squeeze()
        
        if manual_extent is not None and len(manual_extent) == 2 and all(len(sublist) == 2 for sublist in manual_extent):
            map_extent = [manual_extent[0][1], manual_extent[1][1], manual_extent[0][0], manual_extent[1][0]]
        else:
            data_extent_lon = [float(longitude.min()), float(longitude.max())]
            data_extent_lat = [float(latitude.min()), float(latitude.max())]
            map_extent = data_extent_lon + data_extent_lat
        ax.set_extent(map_extent, crs=ccrs.PlateCarree())
        plot_formatted_ticks(ax, map_extent[:2], map_extent[2:], proj=ccrs.PlateCarree(), fontsize=16, label_left=True, label_right=False, label_bottom=True, label_top=False, gridlines=True)

        plot_threshold_zones(ax, longitude, latitude, mag_depth_avg, mag1, mag2, mag3, mag4, mag5, threshold_legend=True)
        plot_streamlines(ax, longitude, latitude, u_depth_avg, v_depth_avg, density=density)

        if gliders is not None:
            plot_add_gliders(ax, gliders, legend=True)
            glider_legend = ax.get_legend()
            if glider_legend:
                glider_legend.get_frame().set_alpha(0.5)
                glider_legend.get_frame().set_facecolor('white')
                ax.add_artist(glider_legend)

        if show_waypoints:
            plot_glider_route(ax, config)
            
        if compute_optimal_path:
            plot_optimal_path(ax, config, model_depth_average)
        
        if show_qc:
            (y_index, x_index), (lat_index, lon_index) = calculate_gridpoint(model_depth_average, latitude_qc, longitude_qc)
            qc_lon = model_depth_average['lon'].isel(x=x_index, y=y_index).values
            qc_lat = model_depth_average['lat'].isel(x=x_index, y=y_index).values
            circle = Circle((qc_lon, qc_lat), radius=0.25, edgecolor='purple', facecolor='none', linewidth=2, transform=ccrs.PlateCarree(), zorder=95)
            ax.add_patch(circle)

        if show_eez:
            plot_add_eez(ax, config, color='dimgrey', linewidth=3, zorder=90)
        
        try:
            plot_bathymetry(ax, config, model_depth_average, isobath1=-100, isobath2=-1000, downsample="auto", show_legend=False)
            bathymetry_legend = ax.get_legend()
            if bathymetry_legend:
                bathymetry_legend.get_frame().set_alpha(0.5)
                ax.add_artist(bathymetry_legend)
        except:
            print(f"WARNING: Bathymetry contouring was unsuccessful for {model_depth_average.attrs['model_name']}. Using default ocean color instead.")
            ax.add_feature(cfeature.OCEAN, zorder=1)

        ax.add_feature(cfeature.GSHHSFeature(scale='full'), edgecolor="black", facecolor="tan", linewidth=0.25, zorder=90)
        ax.add_feature(cfeature.RIVERS, edgecolor="steelblue", linewidth=0.25, zorder=90)
        ax.add_feature(cfeature.LAKES, edgecolor="black", facecolor="lightsteelblue", linewidth=0.25, zorder=90)
        ax.add_feature(cfeature.BORDERS, edgecolor="black", linewidth=0.25, zorder=90)
        
    fig, axs = plt.subplots(1, num_datasets, subplot_kw={'projection': ccrs.Mercator()}, figsize=(10*num_datasets, 10))
    if num_datasets == 1:
        axs = [axs]

    model_names = []
    for ax, (model_data, depth_average_data, bin_average_data) in zip(axs, valid_datasets):
        model_name = depth_average_data.attrs['model_name']
        model_names.append(model_name)
        plot_threshold(ax, config, depth_average_data, latitude_qc, longitude_qc, density, mag1, mag2, mag3, mag4, mag5, gliders, show_waypoints, show_qc, show_eez, manual_extent, compute_optimal_path)
        ax.set_title(f"{model_name}", fontsize=14, fontweight='bold', pad=20)
    
    title_text = f"Depth Averaged Current Threshold Zones - Depth Range: {config['MISSION']['max_depth']}m"
    model_names_combined = " vs. ".join(model_names)
    format_figure_titles(axs[0], fig, config, datetime_index, model_name=model_names_combined, title=title_text)

    file_datetime = format_save_datetime(datetime_index)
    fig_filename = f"DepthAverageThreshold_{config['MISSION']['max_depth']}m_{file_datetime}.png"
    fig_path = os.path.join(directory, fig_filename)
    fig.savefig(fig_path, dpi=300, bbox_inches='tight')
    plt.close(fig)

    end_time = print_endtime()
    print_runtime(start_time, end_time)


In [12]:
GGS_plot_threshold(
        config_flag,
        sub_directory_plots,
        datetime_index,
        model_datasets,
        latitude_qc=latitude_qc_flag, longitude_qc=longitude_qc_flag,
        density=density_flag,
        mag1=mag1_flag, mag2=mag2_flag, mag3=mag3_flag, mag4=mag4_flag, mag5=mag5_flag,
        gliders=glider_data_flag,
        show_waypoints=show_waypoints_flag, show_eez=show_eez_flag, show_qc=show_qc_flag,
        manual_extent=manual_extent_flag,
        compute_optimal_path=compute_optimal_path_flag
    )


### CREATING THRESHOLD PLOT ###

Start time (UTC): 2024-04-11 20:27:43.202873
Computing optimal path for the mission.
!!!WARNING!!!: This feature is experimental and may not work as expected. Use at your own risk.
New Version
Direct path used from (41.66667, -70.5) to (40.0, -70.666664).
Total mission time (adjusted): [17688631.84835901] seconds
Total mission distance: [6098654.257977845] meters
End time (UTC): 2024-04-11 20:32:33.568124
Run time: 0:04:50.365251


In [ ]:
17688631
